# ソーシャルディスタンスを検知するアプリを作ろう
インテル® OpenVINO™ ツールキットの事前学習済みモデルを使ってソーシャルディスタンスを検知するアプリケーションを作成します。事前学習済みモデルを利用することで簡単にAI機能を含んだアプリケーションを開発できることをご体験ください。

## 第1章 人物検出
まずはじめに動画や画像から人物を物体検出するプログラムを作成します。人物を検出するモデルはOpenVINOの事前学習済みモデルを使用します。

### STEP0. 事前学習済みモデルのダウンロード
今回は'person-detection-retail-0013'というモデルを使用するので、下記のコマンドでOpenVINO Model Zoo（OpenVINOのモデルレポジトリー）よりダウンロードしてきます。

ちなみに、このツールはOpenVINOに付属しているModel Downloaderというツールです。通常事前学習済みモデルをダウンロードする際はこちらを使うのがデフォルトです。

In [ ]:
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013

また、下記コマンドを打つと、OpenVINO Model Zooに格納されているすべての事前学習済みモデルの一覧を表示します。お試しください。
```Bash
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --print_all
```

### STEP1. モジュールのインポート
とりあえずこの後使うもの全部をここでインポートしておきます。

In [ ]:
import sys
import time
import os
import io

import cv2
import numpy as np
from scipy.spatial import distance

import logging as log
from PIL import Image
import PIL

from munkres import Munkres

from openvino.inference_engine import IENetwork, IECore

import IPython.display
from IPython.display import clear_output

### STEP2. 必要なクラスを定義

In [ ]:
#検出したPersonの各種情報を保持しておくためのデータホルダークラス
class DetectedPersonInfo:
    def __init__(self, position, feature=[], id=-1):
        self.pos = position
        self.feature = feature
        self.time = time.monotonic()
        self.id = id

#Personを検出するためのディープラーニング推論をOpenVINOで実行するためのクラス
class PersonDetector:
    
    #検出した物体がPersonであるか否かを判定するための閾値
    THRESHOLD = 0.3
    
    def __init__(self, iecore, model_path):
        #AI model of human recognition　settings
        self.net_h  = net = iecore.read_network(model = model_path+ ".xml", weights = model_path + ".bin") 
        self.input_name_h  = next(iter(self.net_h.inputs))                     
        self.input_shape_h = self.net_h.inputs[self.input_name_h].shape           
        self.out_name_h    = next(iter(self.net_h.outputs))                    
        self.out_shape_h   = self.net_h.outputs[self.out_name_h].shape  
        self.exec_net_h    = iecore.load_network(self.net_h, 'CPU')
    
    def inference_and_get_person_bbox(self, image):
        #AI model "human recognition　settings" inference
        in_frame = cv2.resize(image, (self.input_shape_h[3], self.input_shape_h[2]))
        in_frame = in_frame.transpose((2, 0, 1))
        in_frame = in_frame.reshape(self.input_shape_h)
        res_h = self.exec_net_h.infer(inputs={self.input_name_h: in_frame})
        person_bbox_list = res_h[self.out_name_h][0][0]
        
        detected_person_list = []
        for person_bbox in person_bbox_list:
            probability = person_bbox[2]
            if probability > PersonDetector.THRESHOLD: 
                frame = image
                xmin = abs(int(person_bbox[3] * frame.shape[1]))
                ymin = abs(int(person_bbox[4] * frame.shape[0]))
                xmax = abs(int(person_bbox[5] * frame.shape[1]))
                ymax = abs(int(person_bbox[6] * frame.shape[0]))

                person_image = frame[ymin:ymax, xmin:xmax]   
                detected_person_list.append(DetectedPersonInfo([xmin, ymin, xmax, ymax]))
        return detected_person_list

### STEP3. 各種ユーティリティー関数を定義
- 検出した人物にモザイクをかける関数（プライバシー保護用。ハンズオンでは使いませんが。。。）
- モデルの推論結果を反映した出力画像を生成する関数

In [ ]:
#検出したPersonにモザイク処理をするための関数
def mosaic_area(img, xmin, ymin, xmax, ymax, ratio=0.05):
    dst = img.copy()
    dst[ymin:ymax,xmin:xmax] = mosaic(dst[ymin:ymax,xmin:xmax], ratio)
    return dst

def mosaic(img, ratio=0.05):
    small = cv2.resize(img, None, fx=ratio, fy=ratio, interpolation=cv2.INTER_NEAREST)
    big = cv2.resize(small, img.shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
    return big


#推論結果を反映した画面出力用の画像（フレーム）を作成する関数
def create_output_image(image, detected_person_list, mosaic=False):
    for detected_person in detected_person_list:
        id = detected_person.id
        color = (0, 0, 0)
        xmin, ymin, xmax, ymax = detected_person.pos
        if mosaic:
            image = mosaic_area(image, xmin, ymin, xmax, ymax)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 5)
    font = cv2.FONT_HERSHEY_COMPLEX
    cv2.putText(image, 'PersonDetection', (100, 100), font, 1, (0, 0, 255), 2, cv2.LINE_AA)              
    image = cv2.resize(image, dsize=(600, 360))
    return image

### STEP4. メイン関数を定義

In [ ]:
def main(video_file):

    #Instantiate two AI models
    iecore = IECore()
    detector = PersonDetector(iecore, "intel/person-detection-retail-0013/FP32/person-detection-retail-0013")
   
    #Please enter the name of the video file you want to process
    cap = cv2.VideoCapture(video_file)
    while cv2.waitKey(1) != 27:        
        #Read one frame from the video data
        frame_data = cap.read()
        if frame_data[0] == False:
            return
        
        #Set captured frame
        image = frame_data[1]

        #Detect all person's bounding box in the captured frame
        detected_person_list = detector.inference_and_get_person_bbox(image)

        #Window display processing           
        image = create_output_image(image, detected_person_list)
        #cv2.imshow('Frame', image)
        clear_output(wait=True)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        f = io.BytesIO()
        PIL.Image.fromarray(image).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))
        
    cv2.destroyAllWindows()

### STEP5. 実行

In [ ]:
main('town.mp4')

---

## 第2章 人物認証
次に検出した人物を認証するプログラムを追加します。ひとつ前の人物検出は、あくまでもフレームごとに写っている人物たちをスナップショットで検出して、その結果を同じくフレームごとに出力しているだけです。つまり当該フレームの前後フレームとの依存性がないため、現在のフレームに映っている人物が、次のフレームに映っている人物と同一であることを紐づけることができません。

この人物認証では、その問題を解決するためにフレーム毎に人物の認証を行い、複数フレーム間で同一人物を特定することができます。

### STEP0. 事前学習済みモデルのダウンロード
OpenVINOの事前学習済みモデル "person-reidentification-retail-0287"をダウンロードします。

In [ ]:
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name person-reidentification-retail-0287

### STEP1. 人物認証処理を実行する用のクラスを定義
このモデルが行うことは大きく2つです。
- 前段の人物検出で検出された人物ごとに特徴量を抽出
- その特徴量データをデータベース上で管理し、同一人物か否かを判断

In [ ]:
class PersonIdentifier:
    def __init__(self, iecore, model_path):
        #AI model of personal identification settings
        self.net_p = iecore.read_network(model = model_path + ".xml", weights = model_path + ".bin") 
        self.input_name_p  = next(iter(self.net_p.inputs))                  
        self.input_shape_p = self.net_p.inputs[self.input_name_p].shape        
        self.out_name_p    = next(iter(self.net_p.outputs))                 
        self.out_shape_p   = self.net_p.outputs[self.out_name_p].shape 
        self.exec_net_p    = iecore.load_network(self.net_p, 'CPU')
        
        #Person identification management
        self.id_num = 0
        self.dist_threshold = 1.0
        self.timeout_threshold = 10000
        self.feature_db = []
    
    def inference_and_get_feature(self, obj_img):
        obj_img = cv2.resize(obj_img, (128, 256)) 
        obj_img = obj_img.transpose((2,0,1))
        obj_img = np.expand_dims(obj_img, axis=0)

        #AI model "Personal recognition" inference             
        res_reid = self.exec_net_p.infer(inputs={ self.input_name_p : obj_img}) 
        feature = np.array(res_reid[self.out_name_p]).reshape((256))
        return feature
    
    def register_into_database(self, detected_person_list):
        hangarian = Munkres()
        dist_matrix = [ [ distance.cosine(obj_db.feature, obj_cam.feature) for obj_db in self.feature_db ] for obj_cam in detected_person_list ]
        combination = hangarian.compute(dist_matrix)      
        for idx_obj, idx_db in combination:
            if detected_person_list[idx_obj].id!=-1: 
                continue 
            dist = distance.cosine(detected_person_list[idx_obj].feature, self.feature_db[idx_db].feature)
            if dist < self.dist_threshold:
                self.feature_db[idx_db].time = time.monotonic()            
                detected_person_list[idx_obj].id = self.feature_db[idx_db].id              
        del hangarian
        
        for detected_person in detected_person_list:
            if detected_person.id == -1:
                xmin, ymin, xmax, ymax = detected_person.pos
                detected_person.id = self.id_num
                self.feature_db.append(detected_person)
                self.id_num += 1
        
        for i, db in enumerate(self.feature_db):
            if time.monotonic() - db.time > self.timeout_threshold:
                self.feature_db.pop(i)

### STEP2. 画面表示用の関数を少し修正
人物認証の結果が分かりやすいように画面表示時に検出した人物のBounding BoxにIDを表示するようにします。

In [ ]:
def create_output_image_with_id(image, detected_person_list, mosaic=False):
    for detected_person in detected_person_list:
        id = detected_person.id
        color = (0, 0, 0)
        xmin, ymin, xmax, ymax = detected_person.pos
        if mosaic:
            image = mosaic_area(image, xmin, ymin, xmax, ymax)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 5)
        cv2.putText(image, str(id), (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX, 1.0, color, 1)
    font = cv2.FONT_HERSHEY_COMPLEX
    cv2.putText(image, 'PersonIdentification', (100, 100), font, 1, (0, 0, 255), 2, cv2.LINE_AA)              
    image = cv2.resize(image, dsize=(600, 360))
    return image

### STEP3. メイン関数を修正
人物認証用の処理を追加します。

In [ ]:
def main(video_file):
    #Instantiate two AI models
    iecore = IECore()
    detector = PersonDetector(iecore, "intel/person-detection-retail-0013/FP32/person-detection-retail-0013")
    identifier = PersonIdentifier(iecore, "intel/person-reidentification-retail-0287/FP32/person-reidentification-retail-0287")
       
    #Please enter the name of the video file you want to process
    cap = cv2.VideoCapture(video_file)
    while cv2.waitKey(1) != 27:        
        frame = cap.read()
        if frame[0] == False:
            return
        
        image = frame[1]

        #AI model "human recognition　settings" inference
        detected_person_list = detector.inference_and_get_person_bbox(image)

        for detected_person in detected_person_list:
            xmin, ymin, xmax, ymax = detected_person.pos
            person_image = image[ymin:ymax, xmin:xmax] 
            feature = identifier.inference_and_get_feature(person_image)    
            detected_person.feature = feature

        #Register vectors into the database
        identifier.register_into_database(detected_person_list)
        
        #Window display processing           
        image = create_output_image_with_id(image, detected_person_list)
        #cv2.imshow('Frame', image)
        clear_output(wait=True)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        f = io.BytesIO()
        PIL.Image.fromarray(image).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))
        
    cv2.destroyAllWindows()

### STEP4. 実行

In [ ]:
main('town.mp4')

---

## 第3章 ソーシャルディスタンス検知
これまで人物を検出して、それらの人物を認証し、フレーム間で関連性を持たせるようにプログラムを改良してきました。ようやくソーシャルディスタンスを検出する処理を追加できます。

といっても、この処理にはAIは用いません。つまり、モデルを使いません。モデルが出力した情報をもとに独自ロジックでもってソーシャルディスタンスを検出し、一定時間近い距離にいる人たちに対してアラートを表示します。

### STEP1. ソーシャルディスタンス検知処理用のクラスを定義
前述の通り、AIは使いません。人物検出や人物認証の結果を活用して計算していきます。

In [ ]:
class SocialDistanceViolationJudge:
    '''
    Please custom Social_parameter
    SOCIAL_PARAMETER determines Social_Distance.
    But that's not the actual distance. This is the distance between the x and y coordinates on the screen.
    '''
    SOCIAL_PARAMETER = 50.0
    
    # Threshold to decide whether it violates longer time.
    TIME_THRESHOLD = 30.0

    def __init__(self):
        self.violate_person_id_list = list()
        self.start_time = time.monotonic()
        self.first_time = True
   
    def judge_violation_of_social_distance(self, detected_person_list):
        warning_person_id_set = set() 
        old_warning_person_id_set = set()
        
        centroids = np.array([[(detected_person.pos[2]-detected_person.pos[0])/2+detected_person.pos[0],
                               (detected_person.pos[3]-detected_person.pos[1])/2+detected_person.pos[1]] 
                              for detected_person in detected_person_list])
        D_1 = distance.cdist(centroids, centroids, metric = "euclidean")        
        for i in range(0, D_1.shape[0]):
            for j in range(i+1, D_1.shape[1]):
                if  (D_1[i,j] != 0.0 and D_1[i, j] < SocialDistanceViolationJudge.SOCIAL_PARAMETER):
                    warning_person_id_set.add(detected_person_list[i].id)
                    warning_person_id_set.add(detected_person_list[j].id)                 
        if self.first_time:
            old_warning_person_id_set = warning_person_id_set
            self.first_time = False

        if (time.monotonic() - self.start_time) > SocialDistanceViolationJudge.TIME_THRESHOLD:
            self.start_time =  time.monotonic()
            latest_warning_person_id_set = warning_person_id_set
            if len(list(old_warning_person_id_set & latest_warning_person_id_set)) % 2 == 0:
                self.violate_person_id_list += list(old_warning_person_id_set & latest_warning_person_id_set)
                self.violate_person_id_list = list(set(self.violate_person_id_list))
            old_warning_person_id_set = warning_person_id_set
            
        warning_person_id_list = list(warning_person_id_set)
        
        return warning_person_id_list, self.violate_person_id_list

### STEP2. 画像出力用のユーティリティ関数も少し修正
今回、近い距離に一瞬でもいる人たちのBounding Boxを黄色に、近い距離に一定時間以上いる人たちのBounding Boxを赤色にして表示します。

In [ ]:
#Create output image(frame)
def create_output_image_with_violation(image, detected_person_list, warning_id_list, violate_id_list, mosaic=False):
    for detected_person in detected_person_list:
        id = detected_person.id
        color = (0, 0, 0)
        xmin, ymin, xmax, ymax = detected_person.pos
        if mosaic:
            image = mosaic_area(image, xmin, ymin, xmax, ymax)
        if detected_person.id in warning_id_list:
           color = (0, 255, 0)
        if detected_person.id in violate_id_list:
           color = (0, 0, 255)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 5)
    font = cv2.FONT_HERSHEY_COMPLEX
    cv2.putText(image, str(len(violate_id_list)), (100, 100), font, 1, (0, 0, 255), 2, cv2.LINE_AA)              
    image = cv2.resize(image, dsize=(600, 360))
    return image

### STEP3. メイン関数を修正
ソーシャルディスタンス検知系の処理を追加します。

In [ ]:
def main(video_file):
    
    #Instantiate two AI models
    iecore = IECore()
    detector = PersonDetector(iecore, "intel/person-detection-retail-0013/FP32/person-detection-retail-0013")
    identifier = PersonIdentifier(iecore, "intel/person-reidentification-retail-0287/FP32/person-reidentification-retail-0287")
    
    # ソーシャルディスタンスが近い人たちを検出するオブジェクト
    violation_judger = SocialDistanceViolationJudge()
   
    #Please enter the name of the video file you want to process
    cap = cv2.VideoCapture(video_file)
    while cv2.waitKey(1) != 27:    
        frame = cap.read()
        if frame[0] == False:
            return
        
        image = frame[1]

        #AI model "human recognition　settings" inference
        detected_person_list = detector.inference_and_get_person_bbox(image)

        for detected_person in detected_person_list:
            xmin, ymin, xmax, ymax = detected_person.pos
            person_image = image[ymin:ymax, xmin:xmax] 
            feature = identifier.inference_and_get_feature(person_image)    
            detected_person.feature = feature

        #Register vectors into the database
        identifier.register_into_database(detected_person_list)
       
        #Judging a violation of Social Distance
        warning_id_list, violate_id_list = violation_judger.judge_violation_of_social_distance(detected_person_list)
        
        #Window display processing           
        image = create_output_image_with_violation(image, detected_person_list, warning_id_list, violate_id_list)
        #cv2.imshow('Frame', image)
        clear_output(wait=True)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        f = io.BytesIO()
        PIL.Image.fromarray(image).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))
        
    cv2.destroyAllWindows()

### STEP4. 実行

In [ ]:
main('town.mp4')

おつかれさまでした。ここまででアプリケーションの開発は一旦完了です。

一度[READ.ME](https://github.com/hiouchiy/intel_ai_openvino_hands_on)へお戻りいただき、続く応用編を実行するための準備を行ってください。

---

## 【応用編】OpenVINO Model Serverを使ってモデルをWeb API化する
OpenVINO Model ServerはOpenVINOのモデル（IR形式に変換されたモデル）をWeb APIとしてデプロイするためのWebフレームワークです。似たようなツールにTensorFlow版のTF Servingがありますが、まさにTF ServingのOpenVINO版といっても過言ではなく、APIなど非常に似た設計になっております。したがって、TF Servingベースのコードを非常に少ない工数（場合によっては工数なし）でOpenVINO Model Serverで使用できる形に移行可能です。

OpenVINO Model ServerはDockerイメージとして提供されているので、基本的にはDockerがインストールされていることが前提です。[READ.ME](https://github.com/hiouchiy/intel_ai_openvino_hands_on)にイメージお取得方法及び起動方法が記載されておりますのでまずはそちらを実行したうえで以降の作業を行ってください。

### STEP0. 追加モジュールをインストール

In [ ]:
!pip install tensorflow-serving-api

### STEP1. モジュールのインポート

In [ ]:
import cv2
import datetime
import grpc
import numpy as np
import os
from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc, get_model_metadata_pb2

### STEP2. 人物検出用モデルを実行するクラスをModel Serverへ問い合わせるように作成する
今回は人物検出モデルのみWeb API化しています。なので、PersonDetectorクラスを継承したRemotePersonDetectorクラスを作成し、そこにModel Serverにアクセスするための処理を記述します。ちなみに、Model ServerがサポートしているプロトコルはgRPCおよびHTTPです。デフォルトがgRPCであるため、このプログラムもgRPCのクライアントとして記述してあります。

In [ ]:
#Personを検出するためのディープラーニング推論をOpenVINO Model Sererへ問い合わせて実行するためのクラス
class RemotePersonDetector(PersonDetector):
    
    #検出した物体がPersonであるか否かを判定するための閾値
    THRESHOLD = 0.3
    
    def __init__(self, grpc_address='localhost', grpc_port=9000, model_name='person-detection', model_version=None):
        #Settings for accessing model server
        self.grpc_address = grpc_address
        self.grpc_port = grpc_port
        self.model_name = model_name
        self.model_version = model_version
        channel = grpc.insecure_channel("{}:{}".format(self.grpc_address, self.grpc_port))
        self.stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
        
        # Get input shape info from Model Server
        self.input_name, input_shape, self.output_name, output_shape = self.__get_input_name_and_shape__()
        self.input_height = input_shape[2]
        self.input_width = input_shape[3]
    
    def __get_input_name_and_shape__(self):
        metadata_field = "signature_def"
        request = get_model_metadata_pb2.GetModelMetadataRequest()
        request.model_spec.name = self.model_name
        if self.model_version is not None:
            request.model_spec.version.value = self.model_version
        request.metadata_field.append(metadata_field)

        result = self.stub.GetModelMetadata(request, 10.0) # result includes a dictionary with all model outputs
        input_metadata, output_metadata = self.__get_input_and_output_meta_data__(result)
        input_blob = next(iter(input_metadata.keys()))
        output_blob = next(iter(output_metadata.keys()))
        return input_blob, input_metadata[input_blob]['shape'], output_blob, output_metadata[output_blob]['shape']
    
    def __get_input_and_output_meta_data__(self, response):
        signature_def = response.metadata['signature_def']
        signature_map = get_model_metadata_pb2.SignatureDefMap()
        signature_map.ParseFromString(signature_def.value)
        serving_default = signature_map.ListFields()[0][1]['serving_default']
        serving_inputs = serving_default.inputs
        input_blobs_keys = {key: {} for key in serving_inputs.keys()}
        tensor_shape = {key: serving_inputs[key].tensor_shape
                        for key in serving_inputs.keys()}
        for input_blob in input_blobs_keys:
            inputs_shape = [d.size for d in tensor_shape[input_blob].dim]
            tensor_dtype = serving_inputs[input_blob].dtype
            input_blobs_keys[input_blob].update({'shape': inputs_shape})
            input_blobs_keys[input_blob].update({'dtype': tensor_dtype})
        
        serving_outputs = serving_default.outputs
        output_blobs_keys = {key: {} for key in serving_outputs.keys()}
        tensor_shape = {key: serving_outputs[key].tensor_shape
                        for key in serving_outputs.keys()}
        for output_blob in output_blobs_keys:
            outputs_shape = [d.size for d in tensor_shape[output_blob].dim]
            tensor_dtype = serving_outputs[output_blob].dtype
            output_blobs_keys[output_blob].update({'shape': outputs_shape})
            output_blobs_keys[output_blob].update({'dtype': tensor_dtype})

        return input_blobs_keys, output_blobs_keys
    
    def inference_and_get_person_bbox(self, image):
        # 画像データに対する前処理
        img = cv2.resize(image, (self.input_width, self.input_height))
        img = img.transpose((2, 0, 1))
        img = img.reshape(1, 3, self.input_height, self.input_width)
        img = img.astype(np.float32)
        
        # Model ServerにgRPCでアクセスしてモデルをコール
        request = predict_pb2.PredictRequest()
        request.model_spec.name = self.model_name
        request.inputs[self.input_name].CopyFrom(make_tensor_proto(img, shape=(img.shape)))
        result = self.stub.Predict(request, 10.0) # result includes a dictionary with all model outputs
        res_h = make_ndarray(result.outputs[self.output_name])
        person_bbox_list = res_h[0][0]
        
        # モデルの出力から必要なデータを抽出し、呼び出し元へ返す
        detected_person_list = []
        for person_bbox in person_bbox_list:
            probability = person_bbox[2]
            if probability > PersonDetector.THRESHOLD: 
                frame = image
                xmin = abs(int(person_bbox[3] * frame.shape[1]))
                ymin = abs(int(person_bbox[4] * frame.shape[0]))
                xmax = abs(int(person_bbox[5] * frame.shape[1]))
                ymax = abs(int(person_bbox[6] * frame.shape[0]))

                person_image = frame[ymin:ymax, xmin:xmax]   
                detected_person_list.append(DetectedPersonInfo([xmin, ymin, xmax, ymax]))
        
        return detected_person_list

### STEP3. メイン関数を修正
こちらはRemotePersonDetectorを初期化する以外、特段変更点はありません。
なお、Model Serverは同一のホストOS上で稼働しているはずなので、宛先のアドレスはホストOSのIPアドレスを指定ください。ポート番号は9000を指定していますが、これはModel Serverのデフォルトです。model_nameはModel Server上で稼働しているモデルを識別するための名称です。Model Server起動時にしているものを入力ください。

In [ ]:
def main(video_file):
    
    #Instantiate two AI models
    iecore = IECore()
    # 修正したのこの一行だけ↓
    detector = RemotePersonDetector(grpc_address='192.168.145.33', grpc_port='9000', model_name='person-detection')
    identifier = PersonIdentifier(iecore, "intel/person-reidentification-retail-0287/FP32/person-reidentification-retail-0287")
    
    # ソーシャルディスタンスが近い人たちを検出するオブジェクト
    violation_judger = SocialDistanceViolationJudge()
   
    #Please enter the name of the video file you want to process
    cap = cv2.VideoCapture(video_file)
    while cv2.waitKey(1) != 27:    
        frame = cap.read()
        if frame[0] == False:
            return
        
        image = frame[1]

        #AI model "human recognition　settings" inference
        detected_person_list = detector.inference_and_get_person_bbox(image)

        for detected_person in detected_person_list:
            xmin, ymin, xmax, ymax = detected_person.pos
            person_image = image[ymin:ymax, xmin:xmax] 
            feature = identifier.inference_and_get_feature(person_image)    
            detected_person.feature = feature

        #Register vectors into the database
        identifier.register_into_database(detected_person_list)
       
        #Judging a violation of Social Distance
        warning_id_list, violate_id_list = violation_judger.judge_violation_of_social_distance(detected_person_list)
        
        #Window display processing
        image = create_output_image_with_violation(image, detected_person_list, warning_id_list, violate_id_list)
        #cv2.imshow('Frame', image)
        clear_output(wait=True)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        f = io.BytesIO()
        PIL.Image.fromarray(image).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))
        
    cv2.destroyAllWindows()

### 実行
実行結果は変わらないはずです。ただし、人物検出の推論処理のみModel Server上に送られて実行されています。

In [ ]:
main('people.264')

このようにOpenVINO Model Serverを使うことで、既存のOpenVINOのモデル（事前学習済みモデル、カスタムモデル問わず）を簡単にWeb API化できます。ただし、基本的にはモデルの推論処理のみがAPI化されるので、マイクロサービスのような細かい粒度のサービスを作成する際などに非常に向いております。逆に、前処理／後処理などはクライアント側、または、間にもう1サーバー挟んで行うなどの工夫が必要です。

---

# おしまい！